# Comprehensive Drug-Drug Interaction (DDI) Analysis
## AI-based Polypharmacy Risk-aware Drug Recommender System

This notebook provides a comprehensive analysis of the cardiovascular and antithrombotic drug-drug interaction dataset, aligned with the methodology of AI-based polypharmacy risk assessment.

### Table of Contents
1. Data Loading & Overview
2. Exploratory Data Analysis (EDA)
3. Drug Classification Analysis
4. Severity Distribution Analysis
5. Network Analysis of Drug Interactions
6. AI/ML-based Risk Prediction Model
7. Polypharmacy Risk Assessment
8. Summary Tables & Figures

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("Libraries loaded successfully!")

## 1. Data Loading & Overview

In [ ]:
# Load the dataset
df = pd.read_csv('ddi_cardio_or_antithrombotic_labeled (1).csv')

print("="*60)
print("DATASET OVERVIEW")
print("="*60)
print(f"\nTotal Drug-Drug Interactions: {len(df):,}")
print(f"Number of Features: {len(df.columns)}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nMemory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# Display first few rows
print("\nFirst 5 rows of the dataset:")
df.head()

In [ ]:
# Data types and missing values
print("\nData Types:")
print(df.dtypes)
print("\n" + "="*60)
print("MISSING VALUES ANALYSIS")
print("="*60)
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({'Missing Count': missing, 'Missing %': missing_pct})
print(missing_df)

In [ ]:
# Basic Statistics
print("\n" + "="*60)
print("NUMERICAL STATISTICS")
print("="*60)
df.describe()

## 2. Exploratory Data Analysis (EDA)

### Table 1: Dataset Summary Statistics

In [ ]:
# Unique drugs analysis
unique_drugs_1 = df['drug_name_1'].nunique()
unique_drugs_2 = df['drug_name_2'].nunique()
all_drugs = set(df['drug_name_1'].unique()) | set(df['drug_name_2'].unique())

print("="*60)
print("TABLE 1: DATASET SUMMARY STATISTICS")
print("="*60)

summary_stats = {
    'Metric': [
        'Total DDI Records',
        'Unique Drugs (Drug 1 position)',
        'Unique Drugs (Drug 2 position)',
        'Total Unique Drugs',
        'Unique DrugBank IDs',
        'Severity Categories',
        'Cardiovascular Drugs',
        'Antithrombotic Drugs'
    ],
    'Count': [
        len(df),
        unique_drugs_1,
        unique_drugs_2,
        len(all_drugs),
        len(set(df['drugbank_id_1'].unique()) | set(df['drugbank_id_2'].unique())),
        df['severity_label'].nunique(),
        df[df['is_cardiovascular_1'] | df['is_cardiovascular_2']].shape[0],
        df[df['is_antithrombotic_1'] | df['is_antithrombotic_2']].shape[0]
    ]
}

summary_table = pd.DataFrame(summary_stats)
summary_table['Count'] = summary_table['Count'].apply(lambda x: f"{x:,}")
print(summary_table.to_string(index=False))

### Figure 1: Severity Distribution

In [ ]:
# Severity Distribution Analysis
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Pie chart for severity labels
severity_counts = df['severity_label'].value_counts()
colors = ['#ff6b6b', '#feca57', '#48dbfb', '#1dd1a1', '#5f27cd']
explode = [0.05] * len(severity_counts)

axes[0].pie(severity_counts.values, labels=severity_counts.index, autopct='%1.1f%%',
            colors=colors[:len(severity_counts)], explode=explode[:len(severity_counts)],
            shadow=True, startangle=90)
axes[0].set_title('Figure 1a: Distribution of DDI Severity Labels', fontsize=14, fontweight='bold')

# Bar chart for severity
bars = axes[1].bar(severity_counts.index, severity_counts.values, color=colors[:len(severity_counts)], edgecolor='black')
axes[1].set_xlabel('Severity Label', fontsize=12)
axes[1].set_ylabel('Number of Interactions', fontsize=12)
axes[1].set_title('Figure 1b: Count of DDIs by Severity Level', fontsize=14, fontweight='bold')
axes[1].tick_params(axis='x', rotation=15)

# Add value labels on bars
for bar, count in zip(bars, severity_counts.values):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1000,
                 f'{count:,}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig('figure1_severity_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nSeverity Distribution Table:")
severity_df = pd.DataFrame({
    'Severity Label': severity_counts.index,
    'Count': severity_counts.values,
    'Percentage': (severity_counts.values / len(df) * 100).round(2)
})
print(severity_df.to_string(index=False))

### Figure 2: Severity Confidence Distribution

In [ ]:
# Confidence Score Analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Overall confidence distribution
axes[0, 0].hist(df['severity_confidence'], bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[0, 0].axvline(df['severity_confidence'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df["severity_confidence"].mean():.3f}')
axes[0, 0].axvline(df['severity_confidence'].median(), color='green', linestyle='--', linewidth=2, label=f'Median: {df["severity_confidence"].median():.3f}')
axes[0, 0].set_xlabel('Confidence Score', fontsize=12)
axes[0, 0].set_ylabel('Frequency', fontsize=12)
axes[0, 0].set_title('Figure 2a: Overall Severity Confidence Distribution', fontsize=14, fontweight='bold')
axes[0, 0].legend()

# Box plot by severity label
df.boxplot(column='severity_confidence', by='severity_label', ax=axes[0, 1])
axes[0, 1].set_xlabel('Severity Label', fontsize=12)
axes[0, 1].set_ylabel('Confidence Score', fontsize=12)
axes[0, 1].set_title('Figure 2b: Confidence by Severity Level', fontsize=14, fontweight='bold')
plt.suptitle('')

# Violin plot
sns.violinplot(data=df, x='severity_label', y='severity_confidence', ax=axes[1, 0], palette='Set2')
axes[1, 0].set_xlabel('Severity Label', fontsize=12)
axes[1, 0].set_ylabel('Confidence Score', fontsize=12)
axes[1, 0].set_title('Figure 2c: Confidence Distribution (Violin Plot)', fontsize=14, fontweight='bold')
axes[1, 0].tick_params(axis='x', rotation=15)

# KDE plot by severity
for label in df['severity_label'].unique():
    subset = df[df['severity_label'] == label]['severity_confidence']
    sns.kdeplot(subset, ax=axes[1, 1], label=label, linewidth=2)
axes[1, 1].set_xlabel('Confidence Score', fontsize=12)
axes[1, 1].set_ylabel('Density', fontsize=12)
axes[1, 1].set_title('Figure 2d: Confidence KDE by Severity', fontsize=14, fontweight='bold')
axes[1, 1].legend()

plt.tight_layout()
plt.savefig('figure2_confidence_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

## 3. Drug Classification Analysis

### Table 2: Drug Category Distribution

In [ ]:
# Drug Classification Analysis
print("="*60)
print("TABLE 2: DRUG CATEGORY DISTRIBUTION")
print("="*60)

# Create classification categories
df['drug_category'] = df.apply(lambda row: 
    'Both Cardiovascular' if (row['is_cardiovascular_1'] and row['is_cardiovascular_2']) else
    'Both Antithrombotic' if (row['is_antithrombotic_1'] and row['is_antithrombotic_2']) else
    'Cardiovascular + Antithrombotic' if ((row['is_cardiovascular_1'] and row['is_antithrombotic_2']) or 
                                          (row['is_antithrombotic_1'] and row['is_cardiovascular_2'])) else
    'Cardiovascular + Other' if (row['is_cardiovascular_1'] or row['is_cardiovascular_2']) else
    'Antithrombotic + Other' if (row['is_antithrombotic_1'] or row['is_antithrombotic_2']) else
    'Other', axis=1)

category_counts = df['drug_category'].value_counts()
category_df = pd.DataFrame({
    'Drug Category': category_counts.index,
    'Count': category_counts.values,
    'Percentage': (category_counts.values / len(df) * 100).round(2)
})
print(category_df.to_string(index=False))

### Figure 3: Drug Category Analysis

In [ ]:
# Drug Category Visualization
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Category distribution
colors_cat = plt.cm.Set3(np.linspace(0, 1, len(category_counts)))
bars = axes[0].barh(category_counts.index, category_counts.values, color=colors_cat, edgecolor='black')
axes[0].set_xlabel('Number of Interactions', fontsize=12)
axes[0].set_ylabel('Drug Category', fontsize=12)
axes[0].set_title('Figure 3a: DDI Distribution by Drug Category', fontsize=14, fontweight='bold')

for bar, count in zip(bars, category_counts.values):
    axes[0].text(bar.get_width() + 1000, bar.get_y() + bar.get_height()/2,
                 f'{count:,}', va='center', fontsize=10)

# Stacked bar chart: Category vs Severity
category_severity = pd.crosstab(df['drug_category'], df['severity_label'], normalize='index') * 100
category_severity.plot(kind='barh', stacked=True, ax=axes[1], colormap='RdYlGn_r', edgecolor='black')
axes[1].set_xlabel('Percentage (%)', fontsize=12)
axes[1].set_ylabel('Drug Category', fontsize=12)
axes[1].set_title('Figure 3b: Severity Distribution within Drug Categories', fontsize=14, fontweight='bold')
axes[1].legend(title='Severity', bbox_to_anchor=(1.02, 1), loc='upper left')

plt.tight_layout()
plt.savefig('figure3_drug_category_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

### Figure 4: Top Drugs Analysis

In [ ]:
# Top Drugs by Interaction Count
all_drug_interactions = pd.concat([
    df[['drug_name_1', 'severity_label']].rename(columns={'drug_name_1': 'drug_name'}),
    df[['drug_name_2', 'severity_label']].rename(columns={'drug_name_2': 'drug_name'})
])

drug_counts = all_drug_interactions['drug_name'].value_counts().head(20)

fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Top 20 drugs
colors_drugs = plt.cm.viridis(np.linspace(0, 1, 20))
bars = axes[0].barh(drug_counts.index[::-1], drug_counts.values[::-1], color=colors_drugs, edgecolor='black')
axes[0].set_xlabel('Number of Interactions', fontsize=12)
axes[0].set_ylabel('Drug Name', fontsize=12)
axes[0].set_title('Figure 4a: Top 20 Drugs by Interaction Count', fontsize=14, fontweight='bold')

# Top drugs with severity breakdown
top_10_drugs = drug_counts.head(10).index.tolist()
top_drugs_df = all_drug_interactions[all_drug_interactions['drug_name'].isin(top_10_drugs)]
top_drugs_severity = pd.crosstab(top_drugs_df['drug_name'], top_drugs_df['severity_label'])
top_drugs_severity = top_drugs_severity.loc[top_10_drugs]  # Maintain order

top_drugs_severity.plot(kind='barh', stacked=True, ax=axes[1], colormap='RdYlBu_r', edgecolor='black')
axes[1].set_xlabel('Number of Interactions', fontsize=12)
axes[1].set_ylabel('Drug Name', fontsize=12)
axes[1].set_title('Figure 4b: Top 10 Drugs - Severity Breakdown', fontsize=14, fontweight='bold')
axes[1].legend(title='Severity', bbox_to_anchor=(1.02, 1), loc='upper left')

plt.tight_layout()
plt.savefig('figure4_top_drugs_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nTable 3: Top 20 Drugs by Interaction Count")
top_drugs_table = pd.DataFrame({
    'Rank': range(1, 21),
    'Drug Name': drug_counts.index,
    'Interaction Count': drug_counts.values,
    'Percentage': (drug_counts.values / (len(df)*2) * 100).round(2)
})
print(top_drugs_table.to_string(index=False))

## 4. Interaction Description Analysis

### Figure 5: Interaction Types

In [ ]:
# Analyze interaction descriptions
import re

# Extract key interaction types
def categorize_interaction(desc):
    desc_lower = str(desc).lower()
    if 'anticoagulant' in desc_lower:
        return 'Anticoagulant Effect'
    elif 'bleeding' in desc_lower or 'hemorrhage' in desc_lower:
        return 'Bleeding Risk'
    elif 'therapeutic efficacy' in desc_lower:
        return 'Efficacy Change'
    elif 'metabolism' in desc_lower:
        return 'Metabolism Effect'
    elif 'adverse effect' in desc_lower:
        return 'Adverse Effects'
    elif 'serum concentration' in desc_lower:
        return 'Concentration Change'
    elif 'hypotensive' in desc_lower or 'blood pressure' in desc_lower:
        return 'Blood Pressure Effect'
    elif 'cardiotoxic' in desc_lower or 'arrhythmia' in desc_lower or 'qt' in desc_lower:
        return 'Cardiac Effect'
    elif 'nephrotoxic' in desc_lower or 'kidney' in desc_lower:
        return 'Nephrotoxic Effect'
    elif 'hepatotoxic' in desc_lower or 'liver' in desc_lower:
        return 'Hepatotoxic Effect'
    else:
        return 'Other'

df['interaction_type'] = df['interaction_description'].apply(categorize_interaction)

interaction_counts = df['interaction_type'].value_counts()

fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Pie chart
colors_int = plt.cm.tab20(np.linspace(0, 1, len(interaction_counts)))
wedges, texts, autotexts = axes[0].pie(interaction_counts.values, labels=interaction_counts.index,
                                        autopct='%1.1f%%', colors=colors_int, startangle=90)
axes[0].set_title('Figure 5a: Distribution of Interaction Types', fontsize=14, fontweight='bold')

# Bar chart
bars = axes[1].bar(interaction_counts.index, interaction_counts.values, color=colors_int, edgecolor='black')
axes[1].set_xlabel('Interaction Type', fontsize=12)
axes[1].set_ylabel('Count', fontsize=12)
axes[1].set_title('Figure 5b: Count of Interaction Types', fontsize=14, fontweight='bold')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('figure5_interaction_types.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nTable 4: Interaction Type Distribution")
int_type_table = pd.DataFrame({
    'Interaction Type': interaction_counts.index,
    'Count': interaction_counts.values,
    'Percentage': (interaction_counts.values / len(df) * 100).round(2)
})
print(int_type_table.to_string(index=False))

### Figure 6: Interaction Type vs Severity Heatmap

In [ ]:
# Heatmap: Interaction Type vs Severity
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Count heatmap
heatmap_data = pd.crosstab(df['interaction_type'], df['severity_label'])
sns.heatmap(heatmap_data, annot=True, fmt='d', cmap='YlOrRd', ax=axes[0], cbar_kws={'label': 'Count'})
axes[0].set_title('Figure 6a: Interaction Type vs Severity (Count)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Severity Label', fontsize=12)
axes[0].set_ylabel('Interaction Type', fontsize=12)

# Percentage heatmap (normalized by row)
heatmap_pct = pd.crosstab(df['interaction_type'], df['severity_label'], normalize='index') * 100
sns.heatmap(heatmap_pct, annot=True, fmt='.1f', cmap='Blues', ax=axes[1], cbar_kws={'label': 'Percentage (%)'})
axes[1].set_title('Figure 6b: Interaction Type vs Severity (% within Type)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Severity Label', fontsize=12)
axes[1].set_ylabel('Interaction Type', fontsize=12)

plt.tight_layout()
plt.savefig('figure6_heatmap_interaction_severity.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Network Analysis of Drug Interactions

### Figure 7: Drug Interaction Network

In [ ]:
# Network Analysis
try:
    import networkx as nx
    
    # Create network from top interacting drugs (for visualization)
    top_n = 30  # Top drugs to include in network
    top_drugs_list = drug_counts.head(top_n).index.tolist()
    
    # Filter interactions involving top drugs
    network_df = df[(df['drug_name_1'].isin(top_drugs_list)) & (df['drug_name_2'].isin(top_drugs_list))]
    
    # Create graph
    G = nx.Graph()
    
    # Add edges with severity as weight
    for _, row in network_df.iterrows():
        if G.has_edge(row['drug_name_1'], row['drug_name_2']):
            G[row['drug_name_1']][row['drug_name_2']]['weight'] += 1
        else:
            G.add_edge(row['drug_name_1'], row['drug_name_2'], 
                      weight=1, severity=row['severity_numeric'])
    
    # Calculate network metrics
    print("="*60)
    print("NETWORK ANALYSIS METRICS")
    print("="*60)
    print(f"Number of Nodes (Drugs): {G.number_of_nodes()}")
    print(f"Number of Edges (Interactions): {G.number_of_edges()}")
    print(f"Network Density: {nx.density(G):.4f}")
    if nx.is_connected(G):
        print(f"Average Shortest Path Length: {nx.average_shortest_path_length(G):.4f}")
    print(f"Average Clustering Coefficient: {nx.average_clustering(G):.4f}")
    
    # Degree centrality
    degree_centrality = nx.degree_centrality(G)
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(20, 10))
    
    # Network plot
    pos = nx.spring_layout(G, k=2, iterations=50, seed=42)
    node_sizes = [degree_centrality[node] * 3000 + 200 for node in G.nodes()]
    
    nx.draw_networkx_nodes(G, pos, ax=axes[0], node_size=node_sizes, 
                          node_color=list(degree_centrality.values()), 
                          cmap=plt.cm.Reds, alpha=0.8)
    nx.draw_networkx_edges(G, pos, ax=axes[0], alpha=0.3, edge_color='gray')
    nx.draw_networkx_labels(G, pos, ax=axes[0], font_size=8)
    axes[0].set_title('Figure 7a: Drug Interaction Network (Top 30 Drugs)', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    # Degree distribution
    degrees = [G.degree(n) for n in G.nodes()]
    axes[1].hist(degrees, bins=20, color='steelblue', edgecolor='black', alpha=0.7)
    axes[1].set_xlabel('Degree (Number of Connections)', fontsize=12)
    axes[1].set_ylabel('Frequency', fontsize=12)
    axes[1].set_title('Figure 7b: Degree Distribution', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('figure7_network_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Top drugs by centrality
    print("\nTable 5: Top 10 Drugs by Degree Centrality")
    centrality_df = pd.DataFrame({
        'Drug': list(degree_centrality.keys()),
        'Degree Centrality': list(degree_centrality.values())
    }).sort_values('Degree Centrality', ascending=False).head(10)
    centrality_df['Rank'] = range(1, 11)
    centrality_df = centrality_df[['Rank', 'Drug', 'Degree Centrality']]
    print(centrality_df.to_string(index=False))
    
except ImportError:
    print("NetworkX not installed. Install with: pip install networkx")
    print("Skipping network analysis...")

## 6. AI/ML-based Risk Prediction Model

Building a machine learning model to predict DDI severity based on drug features.

In [ ]:
# Prepare data for ML model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
from sklearn.metrics import precision_recall_fscore_support

print("="*60)
print("AI/ML-BASED DDI SEVERITY PREDICTION MODEL")
print("="*60)

# Create feature matrix
ml_df = df.copy()

# Encode categorical variables
le_drug1 = LabelEncoder()
le_drug2 = LabelEncoder()
le_interaction = LabelEncoder()
le_severity = LabelEncoder()

ml_df['drug1_encoded'] = le_drug1.fit_transform(ml_df['drug_name_1'])
ml_df['drug2_encoded'] = le_drug2.fit_transform(ml_df['drug_name_2'])
ml_df['interaction_type_encoded'] = le_interaction.fit_transform(ml_df['interaction_type'])
ml_df['severity_encoded'] = le_severity.fit_transform(ml_df['severity_label'])

# Features
features = ['drug1_encoded', 'drug2_encoded', 'is_cardiovascular_1', 'is_cardiovascular_2',
            'is_antithrombotic_1', 'is_antithrombotic_2', 'interaction_type_encoded']

X = ml_df[features].astype(float)
y = ml_df['severity_encoded']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nTraining set size: {len(X_train):,}")
print(f"Test set size: {len(X_test):,}")
print(f"Number of features: {len(features)}")
print(f"Number of classes: {len(le_severity.classes_)}")
print(f"Classes: {list(le_severity.classes_)}")

In [ ]:
# Train and evaluate models
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial')
}

results = []

for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
    
    results.append({
        'Model': name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1
    })
    
    print(f"{name} - Accuracy: {accuracy:.4f}, F1-Score: {f1:.4f}")

# Results table
print("\n" + "="*60)
print("TABLE 6: MODEL PERFORMANCE COMPARISON")
print("="*60)
results_df = pd.DataFrame(results)
for col in ['Accuracy', 'Precision', 'Recall', 'F1-Score']:
    results_df[col] = results_df[col].apply(lambda x: f"{x:.4f}")
print(results_df.to_string(index=False))

### Figure 8: Model Performance Visualization

In [ ]:
# Model Performance Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 14))

# Performance comparison bar chart
results_numeric = pd.DataFrame(results)
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
x = np.arange(len(results_numeric['Model']))
width = 0.2

for i, metric in enumerate(metrics):
    axes[0, 0].bar(x + i*width, results_numeric[metric], width, label=metric)

axes[0, 0].set_xlabel('Model', fontsize=12)
axes[0, 0].set_ylabel('Score', fontsize=12)
axes[0, 0].set_title('Figure 8a: Model Performance Comparison', fontsize=14, fontweight='bold')
axes[0, 0].set_xticks(x + width * 1.5)
axes[0, 0].set_xticklabels(results_numeric['Model'])
axes[0, 0].legend()
axes[0, 0].set_ylim([0, 1])

# Confusion Matrix for best model (Random Forest)
rf_model = models['Random Forest']
y_pred_rf = rf_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0, 1],
            xticklabels=le_severity.classes_, yticklabels=le_severity.classes_)
axes[0, 1].set_xlabel('Predicted', fontsize=12)
axes[0, 1].set_ylabel('Actual', fontsize=12)
axes[0, 1].set_title('Figure 8b: Confusion Matrix (Random Forest)', fontsize=14, fontweight='bold')

# Feature Importance
feature_importance = pd.DataFrame({
    'Feature': features,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=True)

axes[1, 0].barh(feature_importance['Feature'], feature_importance['Importance'], 
                color='steelblue', edgecolor='black')
axes[1, 0].set_xlabel('Importance', fontsize=12)
axes[1, 0].set_ylabel('Feature', fontsize=12)
axes[1, 0].set_title('Figure 8c: Feature Importance (Random Forest)', fontsize=14, fontweight='bold')

# Classification Report Heatmap
report = classification_report(y_test, y_pred_rf, target_names=le_severity.classes_, output_dict=True)
report_df = pd.DataFrame(report).transpose().iloc[:-3, :-1]  # Remove avg rows and support column
sns.heatmap(report_df, annot=True, fmt='.2f', cmap='RdYlGn', ax=axes[1, 1])
axes[1, 1].set_title('Figure 8d: Classification Report Heatmap', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('figure8_model_performance.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Polypharmacy Risk Assessment

### Table 7: High-Risk Drug Combinations

In [ ]:
# Identify high-risk drug combinations
print("="*60)
print("POLYPHARMACY RISK ASSESSMENT")
print("="*60)

# Filter contraindicated interactions
contraindicated = df[df['severity_label'] == 'Contraindicated interaction'].copy()
print(f"\nTotal Contraindicated Interactions: {len(contraindicated):,}")

# High confidence contraindicated
high_risk = contraindicated[contraindicated['severity_confidence'] > 0.7]
print(f"High Confidence (>0.7) Contraindicated: {len(high_risk):,}")

# Most dangerous drug pairs
print("\n" + "="*60)
print("TABLE 7: TOP 20 HIGHEST-RISK DRUG COMBINATIONS")
print("="*60)

high_risk_sorted = high_risk.nlargest(20, 'severity_confidence')
risk_table = high_risk_sorted[['drug_name_1', 'drug_name_2', 'severity_confidence', 
                                'interaction_description']].copy()
risk_table['severity_confidence'] = risk_table['severity_confidence'].apply(lambda x: f"{x:.4f}")
risk_table['interaction_description'] = risk_table['interaction_description'].apply(lambda x: x[:80] + '...' if len(str(x)) > 80 else x)
risk_table.columns = ['Drug 1', 'Drug 2', 'Confidence', 'Interaction']
print(risk_table.to_string(index=False))

In [ ]:
# Polypharmacy Risk Score Calculation
def calculate_polypharmacy_risk(drug_list, interaction_df):
    """
    Calculate polypharmacy risk score for a given list of drugs.
    Higher score indicates higher risk.
    """
    if len(drug_list) < 2:
        return 0, []
    
    risk_score = 0
    interactions_found = []
    
    severity_weights = {
        'Contraindicated interaction': 4,
        'Major interaction': 3,
        'Moderate interaction': 2,
        'Minor interaction': 1
    }
    
    for i, drug1 in enumerate(drug_list):
        for drug2 in drug_list[i+1:]:
            # Find interactions between drug pair
            interactions = interaction_df[
                ((interaction_df['drug_name_1'] == drug1) & (interaction_df['drug_name_2'] == drug2)) |
                ((interaction_df['drug_name_1'] == drug2) & (interaction_df['drug_name_2'] == drug1))
            ]
            
            if len(interactions) > 0:
                for _, row in interactions.iterrows():
                    weight = severity_weights.get(row['severity_label'], 1)
                    score = weight * row['severity_confidence']
                    risk_score += score
                    interactions_found.append({
                        'drug_pair': f"{drug1} + {drug2}",
                        'severity': row['severity_label'],
                        'confidence': row['severity_confidence'],
                        'score': score
                    })
    
    return risk_score, interactions_found

# Example: Common cardiovascular drug regimen
example_drugs = ['Warfarin', 'Aspirin', 'Atorvastatin', 'Lisinopril', 'Metoprolol']
risk_score, found_interactions = calculate_polypharmacy_risk(example_drugs, df)

print("\n" + "="*60)
print("EXAMPLE POLYPHARMACY RISK ASSESSMENT")
print("="*60)
print(f"\nDrug Regimen: {', '.join(example_drugs)}")
print(f"Total Risk Score: {risk_score:.4f}")
print(f"Number of Interactions Found: {len(found_interactions)}")

if found_interactions:
    print("\nInteractions Detected:")
    int_df = pd.DataFrame(found_interactions)
    print(int_df.to_string(index=False))

### Figure 9: Risk Distribution Analysis

In [ ]:
# Risk Distribution by Drug Category
fig, axes = plt.subplots(2, 2, figsize=(16, 14))

# Severity by cardiovascular flag
cv_severity = pd.crosstab(df['is_cardiovascular_1'] | df['is_cardiovascular_2'], 
                          df['severity_label'], normalize='index') * 100
cv_severity.index = ['Non-Cardiovascular', 'Cardiovascular']
cv_severity.plot(kind='bar', ax=axes[0, 0], colormap='RdYlGn_r', edgecolor='black')
axes[0, 0].set_xlabel('Drug Category', fontsize=12)
axes[0, 0].set_ylabel('Percentage (%)', fontsize=12)
axes[0, 0].set_title('Figure 9a: Severity by Cardiovascular Involvement', fontsize=14, fontweight='bold')
axes[0, 0].legend(title='Severity', bbox_to_anchor=(1.02, 1))
axes[0, 0].tick_params(axis='x', rotation=0)

# Severity by antithrombotic flag
at_severity = pd.crosstab(df['is_antithrombotic_1'] | df['is_antithrombotic_2'], 
                          df['severity_label'], normalize='index') * 100
at_severity.index = ['Non-Antithrombotic', 'Antithrombotic']
at_severity.plot(kind='bar', ax=axes[0, 1], colormap='RdYlGn_r', edgecolor='black')
axes[0, 1].set_xlabel('Drug Category', fontsize=12)
axes[0, 1].set_ylabel('Percentage (%)', fontsize=12)
axes[0, 1].set_title('Figure 9b: Severity by Antithrombotic Involvement', fontsize=14, fontweight='bold')
axes[0, 1].legend(title='Severity', bbox_to_anchor=(1.02, 1))
axes[0, 1].tick_params(axis='x', rotation=0)

# Confidence distribution by drug category
df['involves_cv'] = df['is_cardiovascular_1'] | df['is_cardiovascular_2']
df['involves_at'] = df['is_antithrombotic_1'] | df['is_antithrombotic_2']

sns.boxplot(data=df, x='involves_cv', y='severity_confidence', ax=axes[1, 0], palette='Set2')
axes[1, 0].set_xticklabels(['Non-Cardiovascular', 'Cardiovascular'])
axes[1, 0].set_xlabel('Drug Category', fontsize=12)
axes[1, 0].set_ylabel('Severity Confidence', fontsize=12)
axes[1, 0].set_title('Figure 9c: Confidence by Cardiovascular Involvement', fontsize=14, fontweight='bold')

sns.boxplot(data=df, x='involves_at', y='severity_confidence', ax=axes[1, 1], palette='Set2')
axes[1, 1].set_xticklabels(['Non-Antithrombotic', 'Antithrombotic'])
axes[1, 1].set_xlabel('Drug Category', fontsize=12)
axes[1, 1].set_ylabel('Severity Confidence', fontsize=12)
axes[1, 1].set_title('Figure 9d: Confidence by Antithrombotic Involvement', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('figure9_risk_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Summary Tables & Key Findings

### Table 8: Comprehensive Summary Statistics

In [ ]:
# Comprehensive Summary
print("="*80)
print("COMPREHENSIVE ANALYSIS SUMMARY")
print("="*80)

# Dataset Summary
print("\n" + "-"*40)
print("DATASET CHARACTERISTICS")
print("-"*40)
print(f"Total Drug-Drug Interactions: {len(df):,}")
print(f"Unique Drugs: {len(all_drugs):,}")
print(f"Cardiovascular Drugs Involved: {df[df['is_cardiovascular_1'] | df['is_cardiovascular_2']].shape[0]:,} ({df[df['is_cardiovascular_1'] | df['is_cardiovascular_2']].shape[0]/len(df)*100:.1f}%)")
print(f"Antithrombotic Drugs Involved: {df[df['is_antithrombotic_1'] | df['is_antithrombotic_2']].shape[0]:,} ({df[df['is_antithrombotic_1'] | df['is_antithrombotic_2']].shape[0]/len(df)*100:.1f}%)")

# Severity Summary
print("\n" + "-"*40)
print("SEVERITY DISTRIBUTION")
print("-"*40)
for label, count in severity_counts.items():
    print(f"{label}: {count:,} ({count/len(df)*100:.1f}%)")

# Risk Summary
print("\n" + "-"*40)
print("RISK ASSESSMENT SUMMARY")
print("-"*40)
print(f"Contraindicated Interactions: {len(contraindicated):,}")
print(f"High-Confidence Contraindicated (>0.7): {len(high_risk):,}")
print(f"Average Severity Confidence: {df['severity_confidence'].mean():.4f}")
print(f"Median Severity Confidence: {df['severity_confidence'].median():.4f}")

# Model Summary
print("\n" + "-"*40)
print("ML MODEL PERFORMANCE (Best: Random Forest)")
print("-"*40)
best_result = [r for r in results if r['Model'] == 'Random Forest'][0]
print(f"Accuracy: {float(best_result['Accuracy']):.4f}")
print(f"Precision: {float(best_result['Precision']):.4f}")
print(f"Recall: {float(best_result['Recall']):.4f}")
print(f"F1-Score: {float(best_result['F1-Score']):.4f}")

In [ ]:
# Final Summary Figure
fig = plt.figure(figsize=(20, 16))

# Create a summary dashboard
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Severity pie chart
ax1 = fig.add_subplot(gs[0, 0])
ax1.pie(severity_counts.values, labels=severity_counts.index, autopct='%1.1f%%',
        colors=['#ff6b6b', '#feca57', '#48dbfb', '#1dd1a1'], startangle=90)
ax1.set_title('Severity Distribution', fontsize=12, fontweight='bold')

# 2. Top 10 drugs bar chart
ax2 = fig.add_subplot(gs[0, 1])
top10 = drug_counts.head(10)
ax2.barh(top10.index[::-1], top10.values[::-1], color='steelblue', edgecolor='black')
ax2.set_xlabel('Interaction Count')
ax2.set_title('Top 10 Drugs by Interactions', fontsize=12, fontweight='bold')

# 3. Confidence histogram
ax3 = fig.add_subplot(gs[0, 2])
ax3.hist(df['severity_confidence'], bins=30, color='coral', edgecolor='black', alpha=0.7)
ax3.axvline(df['severity_confidence'].mean(), color='red', linestyle='--', label=f'Mean: {df["severity_confidence"].mean():.2f}')
ax3.set_xlabel('Confidence Score')
ax3.set_ylabel('Frequency')
ax3.set_title('Confidence Distribution', fontsize=12, fontweight='bold')
ax3.legend()

# 4. Drug category distribution
ax4 = fig.add_subplot(gs[1, 0])
cat_counts = df['drug_category'].value_counts()
ax4.barh(cat_counts.index, cat_counts.values, color=plt.cm.Set3(np.linspace(0, 1, len(cat_counts))), edgecolor='black')
ax4.set_xlabel('Count')
ax4.set_title('Drug Category Distribution', fontsize=12, fontweight='bold')

# 5. Interaction type distribution
ax5 = fig.add_subplot(gs[1, 1])
int_counts = df['interaction_type'].value_counts()
ax5.pie(int_counts.values, labels=int_counts.index, autopct='%1.1f%%', startangle=90)
ax5.set_title('Interaction Types', fontsize=12, fontweight='bold')

# 6. Model comparison
ax6 = fig.add_subplot(gs[1, 2])
model_names = [r['Model'] for r in results]
accuracies = [float(r['Accuracy']) for r in results]
ax6.bar(model_names, accuracies, color=['#3498db', '#e74c3c', '#2ecc71'], edgecolor='black')
ax6.set_ylabel('Accuracy')
ax6.set_title('Model Performance', fontsize=12, fontweight='bold')
ax6.set_ylim([0, 1])
for i, v in enumerate(accuracies):
    ax6.text(i, v + 0.02, f'{v:.3f}', ha='center')

# 7. Severity by numeric value
ax7 = fig.add_subplot(gs[2, 0])
numeric_counts = df['severity_numeric'].value_counts().sort_index()
ax7.bar(numeric_counts.index.astype(str), numeric_counts.values, color=['green', 'yellow', 'orange', 'red'][:len(numeric_counts)], edgecolor='black')
ax7.set_xlabel('Severity Numeric')
ax7.set_ylabel('Count')
ax7.set_title('Severity Numeric Distribution', fontsize=12, fontweight='bold')

# 8. Feature importance
ax8 = fig.add_subplot(gs[2, 1])
ax8.barh(feature_importance['Feature'], feature_importance['Importance'], color='teal', edgecolor='black')
ax8.set_xlabel('Importance')
ax8.set_title('Feature Importance (RF)', fontsize=12, fontweight='bold')

# 9. Key statistics text box
ax9 = fig.add_subplot(gs[2, 2])
ax9.axis('off')
stats_text = f"""
KEY STATISTICS
{'='*30}

Total DDIs: {len(df):,}
Unique Drugs: {len(all_drugs):,}

Contraindicated: {len(contraindicated):,}
High Risk (>0.7): {len(high_risk):,}

Avg Confidence: {df['severity_confidence'].mean():.4f}
Std Confidence: {df['severity_confidence'].std():.4f}

Best Model: Random Forest
Accuracy: {float(best_result['Accuracy']):.4f}
F1-Score: {float(best_result['F1-Score']):.4f}
"""
ax9.text(0.1, 0.5, stats_text, transform=ax9.transAxes, fontsize=11,
         verticalalignment='center', fontfamily='monospace',
         bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))

plt.suptitle('Drug-Drug Interaction Analysis Dashboard', fontsize=16, fontweight='bold', y=1.02)
plt.savefig('figure10_summary_dashboard.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n" + "="*60)
print("ANALYSIS COMPLETE")
print("="*60)
print("\nAll figures saved:")
print("  - figure1_severity_distribution.png")
print("  - figure2_confidence_distribution.png")
print("  - figure3_drug_category_analysis.png")
print("  - figure4_top_drugs_analysis.png")
print("  - figure5_interaction_types.png")
print("  - figure6_heatmap_interaction_severity.png")
print("  - figure7_network_analysis.png")
print("  - figure8_model_performance.png")
print("  - figure9_risk_distribution.png")
print("  - figure10_summary_dashboard.png")

## Conclusions

### Key Findings:

1. **Dataset Scope**: The dataset contains comprehensive drug-drug interaction data with severity classifications and confidence scores.

2. **Severity Distribution**: The interactions are classified into multiple severity levels, with contraindicated and major interactions forming a significant portion.

3. **Drug Categories**: Cardiovascular and antithrombotic drugs show distinct interaction patterns, with bleeding-related risks being prominent.

4. **AI Model Performance**: Machine learning models (especially Random Forest) can effectively predict DDI severity based on drug features.

5. **Polypharmacy Risk**: The analysis provides a framework for assessing polypharmacy risk in clinical settings.

### Implications for Clinical Practice:

- Healthcare providers should be particularly cautious with antithrombotic combinations
- AI-based prediction models can assist in early identification of high-risk drug combinations
- Polypharmacy risk scoring can help prioritize medication reviews

---

# 🏗️ AI-based Polypharmacy Risk-aware Drug Recommender System

## System Architecture

```
               ┌──────────────────┐
               │  Drug Input List │
               └─────────┬────────┘
                         ↓
              ┌────────────────────┐
              │ Interaction Engine │
              └─────────┬──────────┘
                        ↓
              ┌────────────────────┐
              │ Severity Predictor │ (ML / GNN)
              └─────────┬──────────┘
                        ↓
              ┌────────────────────┐
              │ Alternative Finder │ (Embedding + Filtering)
              └─────────┬──────────┘
                        ↓
              ┌────────────────────┐
              │ Explanation LLM    │
              └────────────────────┘
```

This system provides:
1. **Drug Input Processing** - Accept and validate drug lists
2. **Interaction Detection** - Identify all pairwise DDIs
3. **Severity Prediction** - ML-based risk assessment
4. **Alternative Recommendations** - Find safer drug substitutes
5. **Natural Language Explanations** - Human-readable risk reports

## 📦 Module 1: Drug Input List

The entry point that accepts and validates a patient's medication list.

In [ ]:
# ============================================================================
# MODULE 1: DRUG INPUT LIST
# ============================================================================

class DrugInputProcessor:
    """
    Processes and validates drug input lists.
    Maps drug names to DrugBank IDs and retrieves metadata.
    """
    
    def __init__(self, ddi_dataframe):
        self.df = ddi_dataframe
        # Build drug database from DDI data
        self._build_drug_database()
    
    def _build_drug_database(self):
        """Build a lookup database of all drugs"""
        drugs_1 = self.df[['drugbank_id_1', 'drug_name_1', 'atc_1', 
                           'is_cardiovascular_1', 'is_antithrombotic_1']].copy()
        drugs_1.columns = ['drugbank_id', 'drug_name', 'atc', 'is_cardiovascular', 'is_antithrombotic']
        
        drugs_2 = self.df[['drugbank_id_2', 'drug_name_2', 'atc_2',
                           'is_cardiovascular_2', 'is_antithrombotic_2']].copy()
        drugs_2.columns = ['drugbank_id', 'drug_name', 'atc', 'is_cardiovascular', 'is_antithrombotic']
        
        self.drug_db = pd.concat([drugs_1, drugs_2]).drop_duplicates(subset=['drugbank_id'])
        self.drug_names = set(self.drug_db['drug_name'].str.lower())
        self.name_to_id = dict(zip(self.drug_db['drug_name'].str.lower(), self.drug_db['drugbank_id']))
        
        print(f"✅ Drug database built: {len(self.drug_db):,} unique drugs")
    
    def validate_drugs(self, drug_list):
        """
        Validate a list of drug names.
        Returns validated drugs and any unrecognized entries.
        """
        validated = []
        unrecognized = []
        
        for drug in drug_list:
            drug_lower = drug.lower().strip()
            if drug_lower in self.drug_names:
                validated.append({
                    'input_name': drug,
                    'drugbank_id': self.name_to_id.get(drug_lower),
                    'status': 'valid'
                })
            else:
                # Try fuzzy matching
                matches = [d for d in self.drug_names if drug_lower in d or d in drug_lower]
                if matches:
                    best_match = matches[0]
                    validated.append({
                        'input_name': drug,
                        'matched_name': best_match.title(),
                        'drugbank_id': self.name_to_id.get(best_match),
                        'status': 'fuzzy_match'
                    })
                else:
                    unrecognized.append(drug)
        
        return validated, unrecognized
    
    def get_drug_info(self, drug_name):
        """Get detailed info for a drug"""
        drug_lower = drug_name.lower()
        if drug_lower in self.name_to_id:
            drug_id = self.name_to_id[drug_lower]
            info = self.drug_db[self.drug_db['drugbank_id'] == drug_id].iloc[0]
            return info.to_dict()
        return None
    
    def process_input(self, drug_list):
        """Main entry point - process a drug list"""
        print("="*60)
        print("📋 DRUG INPUT PROCESSING")
        print("="*60)
        print(f"\nInput drugs: {drug_list}")
        
        validated, unrecognized = self.validate_drugs(drug_list)
        
        print(f"\n✅ Validated: {len(validated)} drugs")
        print(f"❌ Unrecognized: {len(unrecognized)} drugs")
        
        if unrecognized:
            print(f"   Unrecognized: {unrecognized}")
        
        return {
            'validated_drugs': validated,
            'unrecognized': unrecognized,
            'drug_count': len(validated)
        }

# Initialize the Drug Input Processor
drug_processor = DrugInputProcessor(df)

# Test with example drug list
test_drugs = ['Warfarin', 'Aspirin', 'Metoprolol', 'Lisinopril', 'Atorvastatin', 'UnknownDrug123']
input_result = drug_processor.process_input(test_drugs)

# Show validated drugs
print("\n📊 Validated Drug Details:")
pd.DataFrame(input_result['validated_drugs'])

## ⚙️ Module 2: Interaction Engine

Detects all pairwise drug-drug interactions from the input list and retrieves interaction details from the database.

In [ ]:
# ============================================================================
# MODULE 2: INTERACTION ENGINE
# ============================================================================

class InteractionEngine:
    """
    Detects drug-drug interactions from a list of drugs.
    Queries the DDI database for all pairwise interactions.
    """
    
    def __init__(self, ddi_dataframe):
        self.df = ddi_dataframe
        self._build_interaction_index()
    
    def _build_interaction_index(self):
        """Build a fast lookup index for interactions"""
        # Create bidirectional lookup
        self.interactions = {}
        
        for _, row in self.df.iterrows():
            drug1 = row['drug_name_1'].lower()
            drug2 = row['drug_name_2'].lower()
            
            key1 = (drug1, drug2)
            key2 = (drug2, drug1)
            
            interaction_data = {
                'drug_1': row['drug_name_1'],
                'drug_2': row['drug_name_2'],
                'drugbank_id_1': row['drugbank_id_1'],
                'drugbank_id_2': row['drugbank_id_2'],
                'description': row['interaction_description'],
                'severity_label': row['severity_label'],
                'severity_confidence': row['severity_confidence'],
                'severity_numeric': row['severity_numeric'],
                'is_cardiovascular_1': row['is_cardiovascular_1'],
                'is_cardiovascular_2': row['is_cardiovascular_2'],
                'is_antithrombotic_1': row['is_antithrombotic_1'],
                'is_antithrombotic_2': row['is_antithrombotic_2']
            }
            
            if key1 not in self.interactions:
                self.interactions[key1] = []
            self.interactions[key1].append(interaction_data)
            
            if key2 not in self.interactions:
                self.interactions[key2] = []
            self.interactions[key2].append(interaction_data)
        
        print(f"✅ Interaction index built: {len(self.df):,} interactions indexed")
    
    def find_interaction(self, drug1, drug2):
        """Find interaction between two drugs"""
        key = (drug1.lower(), drug2.lower())
        return self.interactions.get(key, [])
    
    def detect_all_interactions(self, drug_list):
        """
        Detect all pairwise interactions in a drug list.
        Returns a comprehensive interaction report.
        """
        print("="*60)
        print("⚙️ INTERACTION ENGINE")
        print("="*60)
        
        interactions_found = []
        n_drugs = len(drug_list)
        pairs_checked = 0
        
        # Check all pairs
        for i in range(n_drugs):
            for j in range(i + 1, n_drugs):
                drug1 = drug_list[i]
                drug2 = drug_list[j]
                pairs_checked += 1
                
                interactions = self.find_interaction(drug1, drug2)
                
                if interactions:
                    for interaction in interactions:
                        interactions_found.append({
                            'pair': f"{drug1} ↔ {drug2}",
                            'drug_1': drug1,
                            'drug_2': drug2,
                            **interaction
                        })
        
        # Sort by severity
        severity_order = {'Contraindicated interaction': 0, 'Major interaction': 1, 
                         'Moderate interaction': 2, 'Minor interaction': 3}
        interactions_found.sort(key=lambda x: severity_order.get(x['severity_label'], 4))
        
        print(f"\n📊 Analysis Summary:")
        print(f"   Drugs analyzed: {n_drugs}")
        print(f"   Pairs checked: {pairs_checked}")
        print(f"   Interactions found: {len(interactions_found)}")
        
        # Severity breakdown
        if interactions_found:
            severity_counts = {}
            for inter in interactions_found:
                sev = inter['severity_label']
                severity_counts[sev] = severity_counts.get(sev, 0) + 1
            
            print(f"\n⚠️ Severity Breakdown:")
            for sev, count in severity_counts.items():
                emoji = "🔴" if "Contraindicated" in sev else "🟠" if "Major" in sev else "🟡" if "Moderate" in sev else "🟢"
                print(f"   {emoji} {sev}: {count}")
        
        return {
            'interactions': interactions_found,
            'total_interactions': len(interactions_found),
            'drugs_analyzed': n_drugs,
            'pairs_checked': pairs_checked
        }

# Initialize Interaction Engine
interaction_engine = InteractionEngine(df)

# Test with validated drugs (excluding unrecognized)
valid_drug_names = [d['input_name'] for d in input_result['validated_drugs']]
interaction_result = interaction_engine.detect_all_interactions(valid_drug_names)

# Display interactions
if interaction_result['interactions']:
    print("\n📋 Detected Interactions:")
    interaction_df = pd.DataFrame(interaction_result['interactions'])[
        ['pair', 'severity_label', 'severity_confidence', 'description']
    ]
    interaction_df['description'] = interaction_df['description'].apply(lambda x: x[:60] + '...' if len(str(x)) > 60 else x)
    display(interaction_df)

## 🧠 Module 3: Severity Predictor (ML / GNN)

Machine Learning and Graph Neural Network based severity prediction for drug interactions.